In [1]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row

In [9]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

** - 1 change dataframe to vector **  
** - 2 missing values count **   
** - 3 fill missing values **   
** - 4 plot feature point distribution **  
** - 5 normalize the feature **  
** - 6 adjust positive negative ratio **  
** - 7 train model **    
** - 8 evaluate model matric **   
** - 9 [feature engineer](http://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark) **   

In [43]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

def check_p_n_cnt(input_df, col_name, pos):
    tmp_table_name = "check_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    total = ctx.sql("select count(*) as tcnt from %s" % tmp_table_name)
    pos = ctx.sql("select count(*) as pcnt from %s where %s = %s" %(tmp_table_name, col_name, pos))
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])

def adjust_p_n_ratio(input_df, cate_col_name, scale, raw_ratio):
    tmp_table_name = "adjust_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    adjusted_df = ctx.sql("""select * from %s where %s = 1 or (%s = 0 and rand() < %s*%s)"""
                          % (tmp_table_name, cate_col_name, cate_col_name, scale, raw_ratio))
    return adjusted_df

def df_to_vec(input_df, ignore_list=[]):
    assembler = VectorAssembler(
        inputCols=[x for x in input_df.columns if x not in ignore_list],
        outputCol='features')
    return assembler.transform(input_df)

def df_to_vec_2(input_df, ignore_list = []):
    labeled_points = input_df.rdd.map(lambda line:LabeledPoint(line[-1],line[0:-1]))
    return labeled_points

def get_corr_matrix(vec, method="pearson"):
    return vec.stat.corr(vec)

In [4]:
train_numer_df = load_csv_df("train_numeric.csv")
test_numer_df = load_csv_df("test_numeric.csv")

In [7]:
#check_p_n_cnt(train_numer_df, "Response", "1") #6879/1183747
adjusted_train_num = adjust_p_n_ratio(train_numer_df, "Response", 1.2, float(6879)/1183747)
print "pos", adjusted_train_num.filter(adjusted_train_num.Response == 1).count()
print "neg", adjusted_train_num.filter(adjusted_train_num.Response == 0).count()

pos 6879
neg 8304


In [45]:
train_numer_vec = df_to_vec_2(train_numer_df)
train_num_vec.take(3)
first_3 = train_numer_df.take(3)
first_3[0][-1]

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.execution.python.EvaluatePython.takeAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 1 times, most recent failure: Lost task 0.0 in stage 30.0 (TID 216, localhost): org.apache.spark.SparkException: Values to assemble cannot be null.
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$assemble$1.apply(VectorAssembler.scala:159)
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$assemble$1.apply(VectorAssembler.scala:142)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
	at org.apache.spark.ml.feature.VectorAssembler$.assemble(VectorAssembler.scala:142)
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$3.apply(VectorAssembler.scala:98)
	at org.apache.spark.ml.feature.VectorAssembler$$anonfun$3.apply(VectorAssembler.scala:97)
	at org.apac

In [19]:
#train_numer_vec = df_to_vec(train_numer_df, ["Id"])
#train_numer_vec.corr(method="pearson")
ignore_list = ["Response"]
assembler = VectorAssembler(
        inputCols=[x for x in train_numer_df.columns if x not in ignore_list],
        outputCol='features')
#print type(train_numer_vec)
train_numer_vec = assembler.transform(train_numer_df)

In [23]:
train_numer_vec.dtypes

[('Id', 'int'),
 ('L0_S0_F0', 'double'),
 ('L0_S0_F2', 'double'),
 ('L0_S0_F4', 'double'),
 ('L0_S0_F6', 'double'),
 ('L0_S0_F8', 'double'),
 ('L0_S0_F10', 'double'),
 ('L0_S0_F12', 'double'),
 ('L0_S0_F14', 'double'),
 ('L0_S0_F16', 'double'),
 ('L0_S0_F18', 'double'),
 ('L0_S0_F20', 'double'),
 ('L0_S0_F22', 'double'),
 ('L0_S1_F24', 'double'),
 ('L0_S1_F28', 'double'),
 ('L0_S2_F32', 'double'),
 ('L0_S2_F36', 'double'),
 ('L0_S2_F40', 'double'),
 ('L0_S2_F44', 'double'),
 ('L0_S2_F48', 'double'),
 ('L0_S2_F52', 'int'),
 ('L0_S2_F56', 'double'),
 ('L0_S2_F60', 'double'),
 ('L0_S2_F64', 'double'),
 ('L0_S3_F68', 'double'),
 ('L0_S3_F72', 'double'),
 ('L0_S3_F76', 'double'),
 ('L0_S3_F80', 'double'),
 ('L0_S3_F84', 'double'),
 ('L0_S3_F88', 'int'),
 ('L0_S3_F92', 'double'),
 ('L0_S3_F96', 'double'),
 ('L0_S3_F100', 'double'),
 ('L0_S4_F104', 'double'),
 ('L0_S4_F109', 'double'),
 ('L0_S5_F114', 'double'),
 ('L0_S5_F116', 'double'),
 ('L0_S6_F118', 'double'),
 ('L0_S6_F122', 'double'),
